In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn import preprocessing
import segyio
from random import randint
import os, os.path
import re 
from PIL import Image
%matplotlib

Using matplotlib backend: Qt5Agg


In [4]:
plt.rcParams['figure.figsize'] = (2.56, 2.56)
plt.rcParams['figure.dpi'] = 129.95

In [5]:
filenameY = './data/dataY.segy'
filenameR = './data/dataR.segy'
filenameG = './data/dataG.segy'
filenameB = './data/dataB.segy'

In [22]:
seismic_Y = segyio.tools.cube(filenameY)
seismic_R = segyio.tools.cube(filenameR)
seismic_G = segyio.tools.cube(filenameG)
seismic_B = segyio.tools.cube(filenameB)

In [ ]:
velocidade_FWI = seismic_A
amplitude = seismic_R
velocidade_RMS = seismic_G
seismic_B = seismic_B

In [ ]:
print('valores antes da normalização de valores entre 0-1')
print('média e valor maximo para a sísmica Y: ',seismic_Y.mean(),seismic_Y.max(),seismic_Y.min())
print('média e valor maximo para a sísmica R: ',seismic_R.mean(),seismic_R.max(),seismic_R.min())
print('média e valor maximo para a sísmica G: ',seismic_G.mean(),seismic_G.max(),seismic_G.min())
print('média e valor maximo para a sísmica B: ',seismic_B.mean(),seismic_B.max(),seismic_B.min())

In [27]:
media_Y=seismic_Y.mean()
max_Y=seismic_Y.max()
min_Y=seismic_Y.min()
media_R=seismic_R.mean()
max_R=seismic_R.max()
min_R=seismic_R.min()
media_G=seismic_G.mean()
max_G=seismic_G.max()
min_G=seismic_G.min()
media_B=seismic_B.mean()
max_B=seismic_B.max()
min_B=seismic_B.min()
print('média e valor maximo para a sísmica Y: ',seismic_Y.mean(),seismic_Y.max(),seismic_Y.min())
print('média e valor maximo para a sísmica R: ',seismic_R.mean(),seismic_R.max(),seismic_R.min())
print('média e valor maximo para a sísmica G: ',seismic_G.mean(),seismic_G.max(),seismic_G.min())
print('média e valor maximo para a sísmica B: ',seismic_B.mean(),seismic_B.max(),seismic_B.min())

média e valor maximo para a sísmica A:  0.4994746 1.0 0.0
média e valor maximo para a sísmica B:  2565.0999 5411.914 1477.8276


In [25]:
print((seismic_Y.shape==seismic_R.shape)and (seismic_R.shape==seismic_G.shape)and (seismic_G.shape==seismic_B.shape))
print(seismic_Y.shape, seismic_R.shape,seismic_G.shape,seismic_B.shape)
IL,XL,t=seismic_Y.shape
print(IL,XL,t)

True
29 3001 751


In [28]:
# regularizando valores para o range 0-1
seismic_Y=((seismic_Y-min_Y)/(max_Y-min_Y))*255
seismic_R=((seismic_R-min_R)/(max_R-min_R))*255
seismic_G=((seismic_G-min_G)/(max_G-min_G))*255
seismic_B=((seismic_B-min_B)/(max_B-min_B))*255
print('média e valor maximo e mínimo para a sísmica Y: ',seismic_Y.mean(),seismic_Y.max(),seismic_Y.min())
print('média e valor maximo e mínimo para a sísmica R: ',seismic_R.mean(),seismic_R.max(),seismic_R.min())
print('média e valor maximo e mínimo para a sísmica G: ',seismic_G.mean(),seismic_G.max(),seismic_G.min())
print('média e valor maximo e mínimo para a sísmica B: ',seismic_B.mean(),seismic_B.max(),seismic_B.min())

média e valor maximo e mínimo para a sísmica A:  0.4994746 1.0 0.0
média e valor maximo e mínimo para a sísmica B:  0.27637243 1.0 0.0


In [29]:
# Gera pares de imagens aleatoreamente amostradas nos volumes sismicos para treinamento da GAN pix2pix
# loop percorrendo todas as inlines e salvando patches com o nome da IL e a posição do patche começando do canto superior esquerdo
# ILn--> InLine number ILp --> InLine patch

n_imagens=1000
patch=256

for i in range(n_imagens):

    plt.rcParams['figure.figsize'] = (2.56, 2.56)
    plt.rcParams['figure.dpi'] = 132.54
    XLp=randint(0,(XL-patch))
    ILp=randint(0,(IL-patch))
    tp=randint(0,(t-patch))
    ILn=randint(0,IL-1)
    XLn=randint(0,XL-1)
    secaoIL=1#randint(0,1) caso for usar cross lines também
    
    if secaoIL==1:
#       Sismica Y
        Y=seismic_Y[ILn,XLp:XLp+patch,tp:tp+patch]
        img_Y = Y.transpose()
        name='./A/ILn'+str(ILn)+'XLp'+str(XLp)+'tp'+str(tp)+'.png'
        result=Image.fromarray((img_Y).astype(np.uint8))
        result.save(name)         

#       Sismica R G B
        R=seismic_R[ILn,XLp:XLp+patch,tp:tp+patch]
        img_R = R.transpose()
        G=seismic_G[ILn,XLp:XLp+patch,tp:tp+patch]
        img_G = G.transpose()
        B=seismic_B[ILn,XLp:XLp+patch,tp:tp+patch]
        img_B = B.transpose()
        RGB=np.stack((R,G,B))
        name='./B/ILn'+str(ILn)+'XLp'+str(XLp)+'tp'+str(tp)+'.png'
        result=Image.fromarray((img_Y).astype(np.uint8))
        result.save(name)     
        

    else:
#       Sismica 
        Y=seismic_A[ILp:ILp+patch,XLn,tp:tp+patch]
        img_Y = img.transpose()
        fig = plt.figure()
        plt.axis('off')
        plt.imshow(img_A,cmap='gray_r')
        name='./A/XLn'+str(XLn)+'ILp'+str(ILp)+'tp'+str(tp)+'.png'
        fig.savefig(name, bbox_inches='tight', pad_inches=0)
        plt.close(fig)  
        
#       Sismica B        
        img=seismic_B[ILp:ILp+patch,XLn,tp:tp+patch]
        img_B = img.transpose()
        fig = plt.figure()
        plt.axis('off')
        plt.imshow(img_B,cmap='gray_r')
        name='./B/XLn'+str(XLn)+'ILp'+str(ILp)+'tp'+str(tp)+'.png'
        fig.savefig(name, bbox_inches='tight', pad_inches=0)
        plt.close(fig)  
        
#       Combina A e B 
        plt.rcParams['figure.figsize'] = (5.12, 2.56)
        img_AB = np.concatenate([img_A, img_B],1)
        fig = plt.figure()
        plt.axis('off')
        plt.imshow(img_AB,cmap='gray_r')
        name='./AB/XLn'+str(XLn)+'ILp'+str(ILp)+'tp'+str(tp)+'.png'
        fig.savefig(name, bbox_inches='tight', pad_inches=0)
        plt.close(fig)

In [ ]:
# Lê segy e gera figuras ordenadas para teste
filename_test = '../../data/basic/dataY.segy'
seismic_test = segyio.tools.cube(filename_test)
seismic_scale=(seismic_test+32767.0)
seismic_scale=seismic_scale/seismic_scale.max()
IL,XL,t=seismic_test.shape
print(IL,XL,t,seismic_scale.mean(),seismic_scale.max(),seismic_scale.min())

In [ ]:
# Escolha a IL:
ILn=300
p=50
npIL=XL//p #numero de patches por IL
npt=t//p #numero de patches por t
plt.rcParams['figure.figsize'] = (2.56, 2.56)
plt.rcParams['figure.dpi'] = 129.95
v0=0
v1=p
for i in range (npt):
    h0=0
    h1=p
    for k in range (npIL):
        img=seismic_test[ILn,h0:h1,v0:v1]
        img_test = img.transpose()
        fig = plt.figure()
        plt.axis('off')
        plt.imshow(img_test,cmap='gray_r')
        name='./test/ILn'+str(ILn)+'XLp'+str(i)+'tp'+str(k)+'.png'
        fig.savefig(name, bbox_inches='tight', pad_inches=0)
        plt.close(fig)  
        h0+=p
        h1+=p       
    v1+=p
    v0+=p

In [ ]:
# Reconstruindo a imagem a partir dos patches
p=256

IL_output= np.zeros([npt*p,npIL*p,4])

path = "./test/"
valid_images = [".jpg",".gif",".png",".tga"]
dire=os.listdir(path)
dire.sort()
x0=0
x1=0
t0=0
t1=0
for filename in dire:
    ext = os.path.splitext(filename)[1] 
    if ext.lower() in valid_images:
        numbers =re.findall('\d+',filename)
        XLp=int(numbers[1])
        tp=int(numbers[2])
        x0=(p*XLp)
        x1=p+(p*XLp)
        t0=(p*tp)
        t1=p+(p*tp)
        IL_output[x0:x1,t0:t1,:]=mpimg.imread(path+filename)
        

#         print(XLp,tp,x0,x1,t0,t1)
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['figure.dpi'] = 300.0
fig = plt.figure()
plt.axis('off')
plt.imshow(IL_output,cmap='gray_r')
name='./OUT/ILn_out'+str(ILn)+'.png'
fig.savefig(name, bbox_inches='tight', pad_inches=0)
plt.close(fig)